In [1]:
import numpy as np
import pandas as pd
import gc

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [4]:
train["id"] = -1
test["target"] = -1
data = train.append(test)
del train
del test
gc.collect()

36

In [5]:
data["user_id"] = data["msno"].astype("category").cat.codes.copy()
data["item_id"] = data["song_id"].astype("category").cat.codes.copy()
data.drop(["msno", "song_id"], inplace=True)
gc.collect()
train_data = data[data["id"] == -1][["user_id", "item_id", "target"]]
test_data = data[data["id"] != -1][["user_id", "item_id", "id"]]
del data
gc.collect()

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


56

In [6]:
from sklearn.model_selection import train_test_split

train_train, train_validation = train_test_split(train_data, train_size=0.8)
del train_data
gc.collect()

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


12

In [7]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix

sparse_train = coo_matrix((
    train_train["target"].astype(np.float32),
    (
        train_train["user_id"],
        train_train["item_id"]
    )
))
sample_weight = coo_matrix((
    np.ones(train_train.shape[0]),
    (
        train_train["user_id"],
        train_train["item_id"]
    )
))

In [8]:
# функция, которая красиво печатает информацию о разреженных матрицах

def sparse_info(sparse_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [9]:
sparse_info(sparse_train)

Размерности матрицы: (34403, 419839)
Ненулевых элементов в матрице: 5901934
Доля ненулевых элементов: 0.0004086158928292744
Среднее значение ненулевых элементов: 0.503607988357544
Максимальное значение ненулевых элементов: 1.0
Минимальное значение ненулевых элементов: 0.0


In [32]:
from scipy.sparse import hstack, identity

item_features = hstack([
    identity(train_train.shape[0])
])

In [10]:
avg_target = train_train["target"].mean()
print(avg_target)
def get_recs(dataset, item_features):
    return dataset.shape[0] * [avg_target]
train_validation["target"] = avg_target
item_features = None

0.5036079698620824


In [31]:
from sklearn.metrics import roc_auc_score

train_recs = get_recs(train_train, item_features)
print("train AUC: {}".format(roc_auc_score(train_train["target"], train_recs)))
roc_auc_score(train_train["target"], train_recs)
train_train["target"]
# validation_recs = get_recs(train_validation, item_features)
# print("test AUC: {}".format(roc_auc_score(train_validation["target"].values, validation_recs)))

train AUC: 0.5


4986180    0
6787896    0
3806617    1
6504477    0
2319656    0
1395848    1
3535277    1
3760130    0
6288588    1
6521309    0
3751849    1
4912761    0
6542450    1
4555237    1
5914150    1
5544020    0
1692889    1
506871     1
5609599    0
679523     1
6147368    0
351412     0
1205463    1
6035154    1
2361336    1
6842872    0
3368972    1
6199807    1
1290824    1
7038351    0
          ..
5729592    0
7115522    0
6017931    0
6038773    0
1291374    0
5668665    0
5970778    0
2732905    1
877577     1
4015032    1
2934411    0
4259992    1
2872101    1
803516     1
4270406    1
3900694    1
5855987    0
3203889    0
3205757    0
7003113    0
65237      0
1509900    0
200679     1
6713592    0
2842773    0
411576     1
4848011    1
3418947    1
5194055    0
4364306    0
Name: target, Length: 5901934, dtype: int64